# NEFS Technology Division - Algorithmic Trading Project Template

### Description 

As a group, you have been tasked with researching, implementing and testing a profitable Foreign Exchange (Forex) trading strategy. The pair you have been assigned is GBP/USD.

Each group will have a number of Quants, Software Engineers and Product Managers. You will need to work as a team to allocate responsibilities for the duration of the project. e.g. a Product Manager may be incharge of setting up and maintaining the Jira backlog of tickets. 

You will be provided with a dataset that will allow you to develop and backtest your strategy. This dataset will provide you with all of the relevant data for a number of years. Your strategy will then be tested on a different timeframe to determine it's profitability (Profit/Loss). 

It may be useful to begin the task by trying to understand the 'bigger picture' of how each component of the project would come together i.e the strategy itself, the risk management system, the data and your testing systems.

You will be provided with an initial template to set out the fundamental components of your system, but you can feel free to expand on these as you wish. However, you need to be wary that your strategy **MUST** still be able to be backtested through the function included below. Please **DO NOT** modify this function in any way.

We have seperated your teams to include Analysts of different disciplines, skills and interests. You should capitalise on this to allow everyone to contribute to the project in the best way that they can. 

Some example roles you may want to have (you do not have to stick to these) : 

- Software Engineer 
- Quant Researcher 
- Quant Developer
- Algo Engineer
- Data Scientist
- Product Manager 

Information Links: 

- [Investopedia - Forex Trading](https://www.investopedia.com/terms/f/foreign-exchange.asp)
- [Investopedia - Forex (Definitions, How To and Examples)](https://www.investopedia.com/terms/f/forex.asp)
- [CME Group British Pound GBP/USD Overview](https://www.cmegroup.com/markets/fx/g10/british-pound.html)
- [Yahoo Finance GBP/USD Live Charts](https://finance.yahoo.com/quote/GBPUSD=X?p=GBPUSD=X&.tsrc=fin-srch)
- [Yahoo Finance USD/GBP Live Charts](https://finance.yahoo.com/quote/GBP=X?p=GBP=X&.tsrc=fin-srch)

#### Rules: 

Here are some rules to simplify this task. 

- You can only buy your positions in GBP and sell your positions in USD. 
- You can not leverage or borrow money to buy a position (no negative balances). If your wallet goes to 0 then something is wrong. 
- You can not sell a position you do not have (no shorting)
- When selling a position, you must sell all of the lots within that position. You can not off-load part of your position.


**What is a standard lot in Forex?**

A standard lot in Forex is equal to **100,000 currency units**. It’s the standard unit size for traders, whether independent or institutional.

If the GBP/USD exchange rate was $1.3000, one standard lot of the base currency (GBP) would be 130,000 units. This means, at the current price, you’d need 130,000 units of the quote currency (USD) to buy 100,000 units of GBP.

--- 

### Datasets

Under this section you will find that two initial datasets are being loaded to be used for this project. These datasets are: 

- GBP/USD Historical Prices between 01/01/2010 - 31/12/2020 (Daily)
- USD/GBP Historical Prices between 01/01/2010 - 31/12/2020 (Daily)

Through your research, you may also find that other datasets across the internet may prove useful in your strategy e.g. Geopolitical data, other currency pairs etc. Feel free to use such datasets to help improve your strategy if you would like to do so. 

In [1]:
#importing some popular packages that you may find useful

from scipy.stats import norm
import numpy as np
import pandas as pd
import math
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
from enum import Enum
from datetime import datetime
import uuid 
import string
from arch import arch_model

In [2]:
complete_gbp_usd_dataset = pd.read_csv('GBPUSD_2010_2022.csv')
complete_usd_gbp_dataset = pd.read_csv('USDGBP_2010_2022.csv')
complete_usd_gbp_dataset

,Date,Open,High,Low,Close,Adj Close,Volume
0,2010-01-01,0.61797,0.61992,0.617970,0.619850,0.619850,0.0
1,2010-01-04,0.61969,0.62247,0.615800,0.620590,0.620590,0.0
2,2010-01-05,0.62040,0.62617,0.618970,0.625360,0.625360,0.0
3,2010-01-06,0.62525,0.62720,0.622540,0.624070,0.624070,0.0
4,2010-01-07,0.62414,0.62890,0.622720,0.627590,0.627590,0.0
...,...,...,...,...,...,...,...
2865,2020-12-25,0.73694,0.73990,0.734592,0.737898,0.737898,0.0
2866,2020-12-28,0.73672,0.74427,0.736500,0.736790,0.736790,0.0
2867,2020-12-29,0.74263,0.74284,0.739610,0.742610,0.742610,0.0
2868,2020-12-30,0.74087,0.74104,0.734350,0.740746,0.740746,0.0


## Signal

In [3]:
data_ori = pd.DataFrame(complete_gbp_usd_dataset)
# if you havent installed the library (talib) then run the following code (remove the three dots )
'''
url = 'https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files'
ext = '0.4.0-oneiric1_amd64.deb -qO'
!wget $url/libta-lib0_$ext libta.deb
!wget $url/ta-lib0-dev_$ext ta.deb
!dpkg -i libta.deb ta.deb
!pip install ta-lib
'''

####################################################################################################

import talib as tal
signal = 0
sum_signal = [0]*21

data_ori["RSI"] = tal.RSI(data_ori["Open"], timeperiod=14)

# Initialize signal
signal = 0

# Loop over data
for i in range(21, len(data_ori)):
    signal = 0
    # Check RSI condition
    if data_ori["RSI"][i] < 30:
        rsi_condition = True
    elif data_ori["RSI"][i] > 70:
        rsi_condition = False
    else:
        rsi_condition = False
        
    # Generate signal if both conditions are met
    if rsi_condition == True: 
        signal += 1
    if rsi_condition == False:
        signal +=-1
    else: 
        signal +=0
    sum_signal.append(signal)

data_ori['Signal_macd'] = ((sum_signal))

In [ ]:
#try the interest rate formula with close price
#complete_usd_gbp_dataset = pd.DataFrame(pd.read_csv('GBPUSD_2010_2022.csv'))
ppp = pd.DataFrame(pd.read_csv('PPP.csv'))
uk_ppp = (ppp['United Kingdom'])
us_ppp = (ppp['United States'])

gbp_usd_ppp = pd.DataFrame({'ppp':uk_ppp/us_ppp, 'date':ppp['Country Name']})
sig = []
gbp_usd_ppp['date'] = pd.to_datetime(gbp_usd_ppp['date'], format='%Y')
gbp_usd_ppp['date']= gbp_usd_ppp['date'].dt.strftime('%Y-%m-%d')
gbp_usd_ppp['date']= pd.to_datetime(gbp_usd_ppp['date'], format='%Y-%m-%d')
start_date = pd.Timestamp('31/12/2009') + pd.DateOffset(day=31)
end_date = pd.Timestamp('01/02/2023') + pd.DateOffset(day=31)
dates = pd.date_range(start_date, end_date, freq='D')
dates.name = 'date'
gbp_usd_ppp.set_index('date', inplace=True)

gbp_usd_ppp = gbp_usd_ppp.reindex(dates, method='ffill')
gbp_usd_ppp.reset_index(inplace=True)
gbp_usd_ppp.rename(columns={'index': 'date'}, inplace=True)


sig=[]
price = (data_ori['Open'])
for i in range(0,len(testtttt)):
    if gbp_usd_ppp['ppp'][i]<(1/price[i]):
        sig.append(1)
    elif gbp_usd_ppp['ppp'][i]>(1/price[i]):
        sig.append(-1)
    else:
        sig.append(0)
data_ori['Signal_ppp'] = sig

In [6]:
data = pd.DataFrame(complete_gbp_usd_dataset)

data['returns'] = data['Open'].pct_change()
data['returns'].dropna()

dates = data['Date']

y = 100*data['returns'].dropna()   #note this still has the n/a term
# adding new column
data['pc_returns'] = 100*data['returns'].dropna()
data['pc_returns'].fillna(0, inplace=True)
# forecasting vol
rolling_predictions = []
test_size = len(data_ori)-1
 # new class for dates
for i in range(test_size):

    train = data['pc_returns'].iloc[:-(test_size - i)]
    model = arch_model(train, p=2, q=0)
    model_fit = model.fit(disp='off')
    pred = model_fit.forecast(horizon=1, reindex=True)
    rolling_predictions.append(np.sqrt(pred.variance.values[-1, :][0]))

rolling_predictions = pd.Series(rolling_predictions, index=y.index)
data['vol_forecast'] = rolling_predictions
sig = [0]*2

for i in range(0,len(data['vol_forecast'])):
    if data['vol_forecast'][i]> 0.75:
        sig.append(1)
    elif data['vol_forecast'][i]<=0.75:
        sig.append(-1)

data_ori['Sig'] = sig


/Users/chunhimyeung/opt/anaconda3/lib/python3.8/site-packages/arch/univariate/volatility.py:1189: RuntimeWarning: invalid value encountered in double_scalars
  scale = np.mean(resids**2) / (target ** (2.0 / power))
/Users/chunhimyeung/opt/anaconda3/lib/python3.8/site-packages/arch/univariate/base.py:753: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/Users/chunhimyeung/opt/anaconda3/lib/python3.8/site-packages/arch/univariate/base.py:309: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.01926. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 10 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
/Users/chunhimyeung/opt/anaconda3/lib/python3.8

### Portfolio 

Under this section you will find the boiler plate code for this task. This includes a class named 'portfolio' which will house key data around your trades such as your budget (monetary portfolio), net profit/loss, current positions and number of trades executed etc.

Each time you run a backtest of your strategy, this object will be reinstatiated and repopulated with the latest backtest results.

You **MUST NOT** modify this code in any way. If you suspect there is a bug, please let Ahmed know and he will look into resolving it.


## All of the backend code. DO NOT CHANGE THIS SECTION IN ANY WAY.

In [7]:
INITIAL_WALLET_CASH = 5000000
STANDARD_LOT = 100000

class Side(Enum):
    BUY = 'BUY'
    SELL = 'SELL'
    
class Currency(Enum):
    GBP = 'GBP'
    USD = 'USD'
    
class Position:
    def __init__(self,
                 date:datetime,
                 size:int,
                 price,
                 currency:Currency,
                 side:Side):
        
        self._id = str(uuid.uuid4())
        self._date = date
        self._currency = currency
        self._price = price
        self._size = size
        self._side = side
        
class Order:
    def __init__(self,
                 position:Position):
        self._id = str(uuid.uuid4())
        self._position = position
    
    def describe(self):
        return f"<Order id: {self._id} position: {self._position}>"

class Portfolio:
    
    def __init__(self):
        self.cash_wallet = INITIAL_WALLET_CASH  # wallet is in GBP
        self.current_positions = {}
        self.closed_positions = {}
    
    def add_position(self,position):
        self.current_positions[position._id] = position
        self.deduct_from_wallet(self.calculate_deduction(position))
                
    def sell_position(self,position:Position):
        self.current_positions.pop(position._id)
        self.closed_positions[position._id] = position
    
    def get_wallet_value(self):
        return self.cash_wallet
    
    def deduct_from_wallet(self,amount):
        self.cash_wallet -= amount
        
    def add_to_wallet(self,amount):
        self.cash_wallet += amount
        
    def calculate_fees(self,position:Position):
        fee = position._size * 2 #broker fee is £2 per standard lot 
        print("FEE CHARGED: £",fee)
        return fee 
    
    def calculate_total_cost_in_gbp(self,position:Position):
        if(position._currency == Currency.GBP):
            return position._price * (position._size * STANDARD_LOT)
        # elif(position._currency == Currency.USD):
        #     return position._price * current_gbp_price  * (position._size * 100000) # x100000 for standard lot

    def calculate_deduction(self,position):
        return self.calculate_total_cost_in_gbp(position) + self.calculate_fees(position)
    

## You can edit from this point onwards 

In [11]:
## You can implement your strategy in this function.
## Feel free to create additional functions and call them from within the 'strategy' function
## As show in the example below, your function must return a dictionary of orders. 
## These orders will be run through the backtesting function
def strategy(data):
    
    FIXED_LOT_SIZE = 1
    
    orders = {}
    
    for index,row in data.iterrows():
        
        if(row["Sig"]>0) or (row['Signal_macd'] >0) or (row['Signal_ppp']>0):
            
            date = datetime.strptime(str(row["Date"]),'%Y-%m-%d').date()
            
            #if we have money available in the wallet, we can buy
            buy_position = Position(date,FIXED_LOT_SIZE,row.Open,Currency.GBP,Side.BUY)
            buy_order = Order(buy_position)
            orders[buy_order._id] = buy_order
    
        if(row["Sig"] <=0) or (row['Signal_macd'] <=0) or (row['Signal_ppp']<=0):
            date = datetime.strptime(str(row["Date"]),'%Y-%m-%d').date()
            sell_position = Position(date,FIXED_LOT_SIZE,row.Open,Currency.GBP,Side.SELL)
            sell_order = Order(sell_position)
            orders[sell_order._id] = sell_order
   
    return orders

In [12]:
# You can use this function to backtest your strategy 
# The function will call your strategy and then execute the orders in the sequence you made them
# Once all of your orders have been completed, you will be shown your final wallet amount and your Profit/Loss.
def local_backtest_strategy(data):
    
    # Initialising the portfolio. £5,000,000 cash and no positions. 
    portfolio = Portfolio()
    order_queue = strategy(data) #calls your function to get all of the orders
    
    for index,order in order_queue.items():
    
        if(order._position._side == Side.BUY):
            # if you have enough cash, you can make the purchase 
            if portfolio.calculate_total_cost_in_gbp(order._position) <= portfolio.cash_wallet:
                portfolio.add_position(order._position)
                print(f"BUY ORDER FILLED : (ID) {str(order._id).upper()}")
            else: 
                print("BOOK CASH IS INSUFFICIENT")
                
        if(order._position._side == Side.SELL):
            # finding the position we want to sell from our current positions within portfolio
            matching_position  = next((position for position in portfolio.current_positions.values()
                                   if position._side == Side.BUY 
                                   and position._size == order._position._size 
                                   and position._price < order._position._price
                                   and position._date < order._position._date)
                                   ,None)
            
            if(matching_position != None):
                sale_earning = order._position._price * (order._position._size * STANDARD_LOT) 
                portfolio.add_to_wallet(sale_earning)
                portfolio.sell_position(matching_position)
                print(f"SELL ORDER FILLED : (ID) {str(order._id).upper()}")
            else:
                print("NO MATCHING POSITION FOUND")
    
    print("ALL ORDERS HAVE BEEN FILLED.")
    print("BOOK CASH LEVEL:", ("£%.2f" % portfolio.cash_wallet))
    
    pnl = portfolio.cash_wallet - INITIAL_WALLET_CASH 
    
    if(pnl < 0):
        print("TOTAL LOSS",("£%.2f" % abs(pnl)))
    else:
        print("TOTAL PROFIT",("£%.2f" % pnl))

local_backtest_strategy(data)

NO MATCHING POSITION FOUND
NO MATCHING POSITION FOUND
NO MATCHING POSITION FOUND
NO MATCHING POSITION FOUND
NO MATCHING POSITION FOUND
NO MATCHING POSITION FOUND
NO MATCHING POSITION FOUND
NO MATCHING POSITION FOUND
NO MATCHING POSITION FOUND
NO MATCHING POSITION FOUND
NO MATCHING POSITION FOUND
NO MATCHING POSITION FOUND
NO MATCHING POSITION FOUND
NO MATCHING POSITION FOUND
NO MATCHING POSITION FOUND
NO MATCHING POSITION FOUND
NO MATCHING POSITION FOUND
NO MATCHING POSITION FOUND
NO MATCHING POSITION FOUND
NO MATCHING POSITION FOUND
NO MATCHING POSITION FOUND
NO MATCHING POSITION FOUND
NO MATCHING POSITION FOUND
NO MATCHING POSITION FOUND
NO MATCHING POSITION FOUND
NO MATCHING POSITION FOUND
FEE CHARGED: £ 2
BUY ORDER FILLED : (ID) 138A82A4-9B8C-4F4E-B820-C7D1678E120E
NO MATCHING POSITION FOUND
FEE CHARGED: £ 2
BUY ORDER FILLED : (ID) D160D68C-8D40-4298-8383-FCD175FDC8D0
NO MATCHING POSITION FOUND
SELL ORDER FILLED : (ID) 3604E2D6-FD09-440F-9460-F68BF464B683
SELL ORDER FILLED : (ID) F

NO MATCHING POSITION FOUND
NO MATCHING POSITION FOUND
NO MATCHING POSITION FOUND
NO MATCHING POSITION FOUND
NO MATCHING POSITION FOUND
NO MATCHING POSITION FOUND
NO MATCHING POSITION FOUND
NO MATCHING POSITION FOUND
NO MATCHING POSITION FOUND
NO MATCHING POSITION FOUND
NO MATCHING POSITION FOUND
NO MATCHING POSITION FOUND
NO MATCHING POSITION FOUND
NO MATCHING POSITION FOUND
NO MATCHING POSITION FOUND
NO MATCHING POSITION FOUND
NO MATCHING POSITION FOUND
NO MATCHING POSITION FOUND
NO MATCHING POSITION FOUND
NO MATCHING POSITION FOUND
NO MATCHING POSITION FOUND
NO MATCHING POSITION FOUND
NO MATCHING POSITION FOUND
NO MATCHING POSITION FOUND
NO MATCHING POSITION FOUND
NO MATCHING POSITION FOUND
NO MATCHING POSITION FOUND
NO MATCHING POSITION FOUND
NO MATCHING POSITION FOUND
NO MATCHING POSITION FOUND
NO MATCHING POSITION FOUND
NO MATCHING POSITION FOUND
NO MATCHING POSITION FOUND
NO MATCHING POSITION FOUND
NO MATCHING POSITION FOUND
NO MATCHING POSITION FOUND
NO MATCHING POSITION FOUND
N

FEE CHARGED: £ 2
BUY ORDER FILLED : (ID) 2336288C-06D2-4CBB-A974-82180ED6BDEF
SELL ORDER FILLED : (ID) 273CCF86-BF80-4D76-8C37-758FEDA29175
FEE CHARGED: £ 2
BUY ORDER FILLED : (ID) 61B6B7A5-7C61-44E0-BF51-58A79748EBC1
NO MATCHING POSITION FOUND
FEE CHARGED: £ 2
BUY ORDER FILLED : (ID) CB64E34B-3C70-49CF-8BCC-1F5D67C2C0CD
SELL ORDER FILLED : (ID) 9CCF6FBD-05BF-4936-B113-D46B5F748C70
FEE CHARGED: £ 2
BUY ORDER FILLED : (ID) EB99643B-5078-4B22-AFF2-8F732ABFB140
NO MATCHING POSITION FOUND
FEE CHARGED: £ 2
BUY ORDER FILLED : (ID) D27C6392-1FBD-4C69-BA8F-444C97A58A90
SELL ORDER FILLED : (ID) 56267827-0519-49D1-B05E-4462D9800FE3
FEE CHARGED: £ 2
BUY ORDER FILLED : (ID) EA317264-092C-4402-AA34-78BDDD1DF9BF
NO MATCHING POSITION FOUND
FEE CHARGED: £ 2
BUY ORDER FILLED : (ID) AFDE25D9-B7DB-45CB-BFC6-4B80C5ED9ADA
SELL ORDER FILLED : (ID) 6BFF010D-5DBA-429A-9216-F608E481D84F
FEE CHARGED: £ 2
BUY ORDER FILLED : (ID) 8139ADA8-B29F-46F8-B8ED-4588EB0DE847
SELL ORDER FILLED : (ID) F854C3C2-B0D1-4567-A65

SELL ORDER FILLED : (ID) B6EF4227-55BA-4E7E-AE4F-7ED4AE23BF44
FEE CHARGED: £ 2
BUY ORDER FILLED : (ID) 898EA25C-5EDF-4C54-9D33-3EBE1379275E
SELL ORDER FILLED : (ID) 6835BF25-A020-44DA-8070-AF40B2974EE2
SELL ORDER FILLED : (ID) 69141DB8-2E54-485C-9BAC-B465F840636F
SELL ORDER FILLED : (ID) D2599BF2-B7B2-4533-84D2-D96DC9EED955
SELL ORDER FILLED : (ID) 9BF38A73-81C5-4D12-989F-A3222CF0286A
SELL ORDER FILLED : (ID) 9CC6FB3E-FB2C-468A-8848-1D641D990222
SELL ORDER FILLED : (ID) 48F46BE9-FEF9-4D24-B748-E8BDB47AA6AF
SELL ORDER FILLED : (ID) 9FB42450-048A-489B-9D0D-B4A3D13847E2
SELL ORDER FILLED : (ID) D214D813-008B-4D94-8460-E921A95C6C29
SELL ORDER FILLED : (ID) 3997D6CC-E3E6-43ED-8CF8-D2CC4E0CDC57
SELL ORDER FILLED : (ID) 90DFDD68-371D-4387-AB6B-7E98EAD0650B
SELL ORDER FILLED : (ID) E4E7204E-3BFB-4395-9FCA-E79F01303D6B
SELL ORDER FILLED : (ID) 14015C0D-D634-4990-A104-4E8ED8979D58
SELL ORDER FILLED : (ID) 9C039E10-866F-4384-A41D-3E8D012C4C8E
FEE CHARGED: £ 2
BUY ORDER FILLED : (ID) 1FF80845-526F

NO MATCHING POSITION FOUND
NO MATCHING POSITION FOUND
FEE CHARGED: £ 2
BUY ORDER FILLED : (ID) 71B06BC8-0A30-4F5B-80EE-27080E64DDAD
NO MATCHING POSITION FOUND
FEE CHARGED: £ 2
BUY ORDER FILLED : (ID) 484305AC-F25C-43DF-9817-A97184A636E3
NO MATCHING POSITION FOUND
FEE CHARGED: £ 2
BUY ORDER FILLED : (ID) 070BAFC5-1AB0-4FDF-AC3B-47E7C0FA91BF
NO MATCHING POSITION FOUND
FEE CHARGED: £ 2
BUY ORDER FILLED : (ID) 8C441734-3198-4902-AB58-1C2817FCA86B
NO MATCHING POSITION FOUND
FEE CHARGED: £ 2
BUY ORDER FILLED : (ID) 4D217F93-CBBA-4691-8641-CD1DF33BFA18
NO MATCHING POSITION FOUND
FEE CHARGED: £ 2
BUY ORDER FILLED : (ID) 40286436-E190-49FB-A056-7017496623EA
NO MATCHING POSITION FOUND
NO MATCHING POSITION FOUND
FEE CHARGED: £ 2
BUY ORDER FILLED : (ID) 678538CC-359A-41D2-B50C-5CA0E3B3696F
NO MATCHING POSITION FOUND
FEE CHARGED: £ 2
BUY ORDER FILLED : (ID) 46BFEAA5-44C3-407E-8B15-7173437B65FC
SELL ORDER FILLED : (ID) 15C13437-12A0-4AEF-9884-DFF17BCA2CF4
FEE CHARGED: £ 2
BUY ORDER FILLED : (ID) FB7

FEE CHARGED: £ 2
BUY ORDER FILLED : (ID) 444B7DCD-08BE-44F0-8603-EC0F74326A41
NO MATCHING POSITION FOUND
FEE CHARGED: £ 2
BUY ORDER FILLED : (ID) 38B99F05-E0DB-4B4D-850D-D2E9E9F42E02
NO MATCHING POSITION FOUND
FEE CHARGED: £ 2
BUY ORDER FILLED : (ID) F8CD2B8C-E76D-4763-9D08-98C43C553505
SELL ORDER FILLED : (ID) D06C68F7-9F61-4D69-BE85-968EC06EBCC1
FEE CHARGED: £ 2
BUY ORDER FILLED : (ID) 0537D597-9E0D-4389-B16B-006072BC896D
NO MATCHING POSITION FOUND
FEE CHARGED: £ 2
BUY ORDER FILLED : (ID) D553BEC6-DC5C-4D93-968F-CF6CDA3F0F48
SELL ORDER FILLED : (ID) 33FA1C05-45AF-4D8A-B8C3-A98F4D2AAAEE
FEE CHARGED: £ 2
BUY ORDER FILLED : (ID) 6B941364-9E2B-4D7D-87BA-84B6B7411B3D
SELL ORDER FILLED : (ID) 5A70513E-BF4C-4811-A376-4FAC2D48FB20
FEE CHARGED: £ 2
BUY ORDER FILLED : (ID) C8482583-37D7-4658-A139-9CA8896E8A47
SELL ORDER FILLED : (ID) 633186CF-98F8-4C2B-9D84-1979323D3715
FEE CHARGED: £ 2
BUY ORDER FILLED : (ID) 9C885989-FFBF-406D-8200-6DB283CCB05D
SELL ORDER FILLED : (ID) F21F0FE3-11EB-4BCA-9D1

## Data Analysis and Research

You can include all of your data analysis and research under this section. Maybe some cool charts and formulas too? Next time.

## Risk Management 

You can include your risk management process and functions (code) under this section.

In [ ]:
def risk_manager():
    pass 